In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path
from plaid.compression.hourglass_vq import HourglassVQLightningModule
import matplotlib.pyplot as plt
import os

In [3]:
pdb_dir = "/homefs/home/lux70/cache/"

from plaid.evaluation import run_tmalign
import numpy as np

def run_tm(orig_pdbs, recons_pdbs):
    all_scores = []
    for orig, recons in zip(orig_pdbs, recons_pdbs):
        tmscore = run_tmalign(orig, recons)
        all_scores.append(tmscore)
    return all_scores

In [4]:
from plaid.constants import COMPRESSION_INPUT_DIMENSIONS, COMPRESSION_SHORTEN_FACTORS
import glob
dim_to_dir = {}

for model_id in COMPRESSION_INPUT_DIMENSIONS.keys():
    if os.path.exists(os.path.join(pdb_dir, model_id)):
        dim = COMPRESSION_INPUT_DIMENSIONS[model_id]
        dirpath = os.path.join(pdb_dir, model_id)

        originals = glob.glob(str(os.path.join(dirpath, "orig*pdb")))
        predicted = glob.glob(str(os.path.join(dirpath, "recons*pdb")))
        originals.sort()
        predicted.sort()

        tm_scores = run_tm(originals, predicted)
        dim_to_dir[dim] = {
            "original": originals,
            "predicted": predicted,
            "tmscores": tm_scores
        }

In [12]:
for k, v in dim_to_dir.items():
    data = dim_to_dir[k]
    tmscores = data['tmscores']
    print(k, f"{tmscores[3]:.3f}")

4 0.789
8 0.920
16 0.968
32 0.995
64 0.991
128 0.999
256 1.000
512 0.997
1024 1.000


In [8]:
data['original'][i]

'/homefs/home/lux70/cache/ich20c3q/orig_pred_11.pdb'

In [7]:
from plaid.utils import view_py3Dmol
import py3Dmol

curdim = 256
data = dim_to_dir[curdim]

for i in range(4):
    print(i, data['tmscores'][i])
    view = py3Dmol.view(width=400, height=300)
    view.addModelsAsFrames(data['original'][i])
    view.addModelsAsFrames(data['predicted'][i])
    view.setStyle({"model": 0}, {"cartoon": {"color": "orange"}})
    view.setStyle({"model": 1}, {"cartoon": {"color": "blue"}})
    view.zoomTo()
    view.show()

0 0.9189


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

1 0.85675


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

2 0.99997


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3 0.99972


3Dmol.js failed to load for some reason. Please check your browser console for error messages.